# Answer Extraction Pipeline for TriviaQA

Contains dataset preprocessing, training and evaluation code for the question answering task using DistilBERT.


In [ ]:
!nvidia-smi

Wed Jun  9 20:04:43 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 466.27       Driver Version: 466.27       CUDA Version: 11.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| 18%   56C    P8    21W / 250W |    758MiB / 11264MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Installs and Imports

In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
!pip install pickle5
import pickle5 as pickle

In [ ]:
import numpy as np
import os
import re
import random

## Run in Google Drive

In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)  

ir_project_drive_folder = "IR Projekt"
data_folder = "/content/drive/My Drive/{}/data/wikipedia".format(ir_project_drive_folder)
models_folder = '/content/drive/My Drive/{}/saved_models'.format(ir_project_drive_folder)

Mounted at /content/drive


## Run Locally

In [ ]:
base_folder = '..'
data_folder = base_folder + '/data'
models_folder = base_folder + '/models'

## Files and Folders

In [ ]:
raw_folder = data_folder + '/raw'
preprocessed_folder = data_folder + '/no-pron/preprocessed'

question_answering_models_folder = models_folder + '/question_answering_task'

# qa filenames
qa_wikipedia_verified_dev_filename = raw_folder + '/qa/verified-wikipedia-dev.json'
qa_wikipedia_dev_filename = raw_folder + '/qa/wikipedia-dev.json'
qa_wikipedia_test_without_answers_filename = raw_folder + '/qa/wikipedia-test-without-answers.json'
qa_wikipedia_train_filename = raw_folder + '/qa/wikipedia-train.json'

# evidence files
wikipedia_evidence_file = raw_folder + '/wikipedia_evidence_dict.pkl'

In [ ]:
def save_as_pickle(obj, filename):
    """
    save an object in a pickle file dump
    :param obj: object to dump
    :param filename: target file
    :return:
    """
    directory = os.path.dirname(filename)
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    with open(filename, 'wb') as file:
        pickle.dump(obj, file, protocol=pickle.HIGHEST_PROTOCOL)


def load_pickle(filename):
    """
    load an object from a given pickle file
    :param filename: source file
    :return: loaded object
    """
    with open(filename, 'rb') as file:
        return pickle.load(file)

## Load Data

In [ ]:
# Evidence data

documents_dict = load_pickle(wikipedia_evidence_file)
print(len(documents_dict))


73930


# Training & Evaluation 

In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
pad_to_max_length = True
pad_on_right = False # tokenizer.padding_side == "right"  # Padding side determines if we do (question|context) or (context|question).
doc_stride = 128
max_seq_length = tokenizer.model_max_length

# Postprocessing Config
version_2_with_negative = True
n_best_size = 20
max_answer_length = 50
null_score_diff_threshold = 0

negative_to_positive = 1

## Import framework code adapted from [Huggingface Transformers Question Answering](https://github.com/huggingface/transformers/tree/v4.6.1/examples/pytorch/question-answering)

In [ ]:
%run -i -n QuestionAnsweringTrainerFramework.py

## Utils

In [ ]:
def read_trivia_entries(groups):
  answers = groups['Answer']
  questions = groups['Question']
  question_ids = groups['QuestionId']
  entity_pages_list = groups['EntityPages']

  result_ids = []
  result_contexts = []
  result_questions = []
  result_answers = []

  for answer, question, question_id, entity_pages in zip(answers, questions, question_ids, entity_pages_list):
    answer_texts = [answer['Value']] + answer['Aliases']
    if 'HumanAnswers' in answer and answer['HumanAnswers'] is not None:
      answer_texts += answer['HumanAnswers']  

    for entity_page in entity_pages:
      document_name = entity_page['Filename']
      document_name_clean = re.sub("[\?:\*\"]", "_", document_name)
      
      if not document_name_clean in documents_dict:
        continue

      context = documents_dict[document_name_clean]

      found_answer_texts = []
      found_answer_starts = []
      for answer_text in answer_texts:
        try:
          answer_start = context.index(answer_text)
          found_answer_texts.append(answer_text)
          found_answer_starts.append(answer_start)

        except ValueError as e:
          pass

      if len(found_answer_texts) == 0:
        continue

      id = question_id + '__' + document_name

      result_ids.append(id)
      result_contexts.append(context)
      result_questions.append(question)
      result_answers.append({'text': found_answer_texts, 'answer_start': found_answer_starts})

  return {"id": result_ids, "context": result_contexts, "question": result_questions, "answer": result_answers}

In [ ]:
from datasets import load_dataset

def create_qa_dataset(path):
  dataset = load_dataset('json', data_files=[path], field='Data', keep_in_memory=False)["train"]
  return dataset.map(read_trivia_entries, batched=True, remove_columns=dataset.column_names, batch_size=32)

## Training

### Training Data

In [ ]:
dataset_name = "distilbert/1positive-to-" + str(negative_to_positive) + "negative"
model_name = "distilbert/full_wikipedia-1to" + str(negative_to_positive) 

In [ ]:
train_base_dataset_directory = data_folder + "/qa-datasets/" + dataset_name + "/train/base"
train_encodings_dataset_directory = data_folder + "/qa-datasets/" + dataset_name + "/train/encodings"

val_verified_base_dataset_directory = data_folder + "/qa-datasets/" + dataset_name + "/val_verified/base"
val_verified_encodings_dataset_directory = data_folder + "/qa-datasets/" + dataset_name + "/val_verified/encodings"

model_directory = question_answering_models_folder + '/' + model_name

#### Create Training Dataset and Store it

In [ ]:
train_base_dataset = create_qa_dataset(qa_wikipedia_train_filename)

train_dataset = train_base_dataset.map(
    prepare_train_features,
    batched = True,
    remove_columns = train_base_dataset.column_names,
    batch_size = 32
)

In [ ]:
train_base_dataset.save_to_disk(train_base_dataset_directory)
train_dataset.save_to_disk(train_encodings_dataset_directory)

#### Load existing Training Dataset

In [ ]:
from datasets import Dataset

train_base_dataset = Dataset.load_from_disk(train_base_dataset_directory)
train_dataset = Dataset.load_from_disk(train_encodings_dataset_directory)

### Evaluation Data

#### Create Evaluation Dataset and Store it

In [ ]:
val_verified_base_dataset = create_qa_dataset(qa_wikipedia_verified_dev_filename)

print(len(val_verified_base_dataset))

val_verified_dataset = val_verified_base_dataset.map(
    prepare_validation_features,
    batched = True,
    remove_columns = val_verified_base_dataset.column_names,
    batch_size = 32
)

In [ ]:
val_verified_base_dataset.save_to_disk(val_verified_base_dataset_directory)
val_verified_dataset.save_to_disk(val_verified_encodings_dataset_directory)

#### Load existing Evaluation Dataset

In [ ]:
from datasets import Dataset

val_verified_base_dataset = Dataset.load_from_disk(val_verified_base_dataset_directory)
val_verified_dataset = Dataset.load_from_disk(val_verified_encodings_dataset_directory)

### Create Metrics

In [ ]:
from datasets import load_metric
from transformers import EvalPrediction

metric = load_metric("squad_v2" if version_2_with_negative else "squad")

def compute_metrics(p: EvalPrediction):
    metrics = metric.compute(predictions = p.predictions, references = p.label_ids, no_answer_threshold=.8)
    metrics["eval_exact"] = metrics["exact"]
    return metrics

In [ ]:
from transformers import DistilBertForQuestionAnswering

model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this mode

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir = model_directory,# output directory
    num_train_epochs = 10,              # total number of training epochs
    per_device_train_batch_size = 16,  # batch size per device during training
    per_device_eval_batch_size = 64,   # batch size for evaluation
    warmup_steps = 500,                # number of warmup steps for learning rate scheduler
    weight_decay = 0.01,               # strength of weight decay
    logging_dir = model_directory + '/logs',            # directory for storing logs
    logging_steps = 50,
 #   save_steps = 1000,
    save_total_limit = 3,
    evaluation_strategy = "steps",
    eval_steps = 1000,
    load_best_model_at_end = True,
    metric_for_best_model = "eval_exact"
)

trainer = QuestionAnsweringTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_verified_dataset,
    eval_examples=val_verified_base_dataset,
    tokenizer=tokenizer,
    post_process_function=post_processing_function,
    compute_metrics=compute_metrics
)

### Run Training

In [ ]:
continue_existing_training = len(os.listdir(model_directory)) > 0

if continue_existing_training:
  print("Checkpoint directory exists. Continuing existing model...")
  train_result = trainer.train(True)
else:
  print("No existing checkpoint found. Starting fresh...")
  train_result = trainer.train()

Checkpoint directory exists. Continuing existing model...


Step,Training Loss,Validation Loss,,Exact,F1,Total,Exact Thresh,F1 Thresh
50000,0.308600,No log,622,47.749196,51.660742,622,0.000000,0.000000
51000,0.355800,No log,622,48.070740,52.049839,622,0.000000,0.000000
52000,0.330100,No log,622,48.553055,52.200310,622,0.000000,0.000000
53000,0.307300,No log,622,46.463023,50.168810,622,0.000000,0.000000
54000,0.344200,No log,622,48.874598,52.925986,622,0.000000,0.000000
55000,0.297900,No log,622,46.623794,50.286238,622,0.000000,0.000000
56000,0.231000,No log,622,46.945338,50.468365,622,0.000000,0.000000
57000,0.271900,No log,622,47.427653,51.080616,622,0.000000,0.000000
58000,0.296700,No log,622,46.945338,51.244241,622,0.000000,0.000000
59000,0.264700,No log,622,46.784566,50.914573,622,0.000000,0.000000


KeyboardInterrupt: 

### Run Evaluation

In [ ]:
from transformers.trainer_utils import get_last_checkpoint

model = DistilBertForQuestionAnswering.from_pretrained(get_last_checkpoint(model_directory))

training_args = TrainingArguments(
   output_dir = model_directory,# output directory
   per_device_eval_batch_size = 64
)

trainer = QuestionAnsweringTrainer(
    model=model,
    args=training_args,
    eval_dataset=val_verified_dataset,
    eval_examples=val_verified_base_dataset,
    tokenizer=tokenizer,
    post_process_function=post_processing_function,
    compute_metrics=compute_metrics
)

In [ ]:
metrics = trainer.evaluate()
metrics